**Exercise 3**

Design and implement a secure channel. Use the following interface:

```
class Peer(object):
def __init__(self, key):
    ...

def send(self, msg):
    ... # protect the message
    return protected_msg # type of protected_msg is ’str’

def receive(self, protected_msg):
    ... # verify the message and print errors if any
    print msg # successfuly recovered plaintext

# Example
alice = Peer("very secret key!")
bob = Peer("very secret key!")
msg1 = alice.send("Msg from alice to bob")
bob.receive(msg1)
msg2 = alice.send("Another msg from alice to bob")
bob.receive(msg2)
msg3 = bob.send("Hello alice")
alice.receive(msg3)
```


In [2]:
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes
import base64


class Peer:
    # session key/ init key
    __session_key: bytes
    # init counters for send/receive message
    __sender_counter: int
    __receiver_counter: int

    def __init__(self, session_key: str) -> None:
        # Reset counter
        self.__sender_counter = 0
        self.__receiver_counter = 0
        # convert the session key into bytes
        self.__session_key = session_key.encode()

    def send(self, msg: str) -> str:
        # protect the message
        try:
            # check the number of counter and throw error if the counter is more than 32 bit
            if self.__sender_counter >= 2**32 - 1:
                raise ValueError("counter overloaded")
            # else increase sending message
            self.__sender_counter += 1

            # in transmitted message, the counter will have 4 bytes, represent for 32 bits number
            sender_msg_counter_to_bytes = self.__sender_counter.to_bytes(4)
            # padding (8 bytes 0)
            padding = b"\00" * 8

            # nonce based on padding + sender messages counter (4 bytes)
            nonce = padding + sender_msg_counter_to_bytes

            # Encrypt the message with CCM Mode
            cipher = AES.new(self.__session_key, AES.MODE_CCM, nonce)
            cipher_text = cipher.encrypt(msg.encode())
            # 16 bytes length authentication tag
            auth_tag = cipher.digest()

            # concatenating into the protected message
            # protected msg = padding + msg counter + encrypted msg + auth_tag
            protected_msg = base64.b64encode(
                padding + sender_msg_counter_to_bytes + cipher_text + auth_tag
            ).decode("utf-8")
            return protected_msg
        except ValueError as error:
            print(f"Error: {error}")

    def receive(self, protected_msg: str) -> str:
        # verify the message and print errors if any
        try:
            # decode the protected message into bytes
            protected_msg_bytes = base64.b64decode(protected_msg)
            # First 8 bytes for padding value
            padding = protected_msg_bytes[:8]
            # Next 4 bytes for message number value
            message_number_in_bytes = protected_msg_bytes[8:12]
            # Next bytes for encrypted message value
            cipher_text = protected_msg_bytes[12:-16]
            # The last 16 bytes for the cipher text value
            tag = protected_msg_bytes[-16:]

            message_number = int.from_bytes(message_number_in_bytes)

            # if received message is not in the right order, throw error
            if message_number <= self.__receiver_counter:
                raise ValueError("Message order incorrect")

            # update the number of receive messages
            self.__receiver_counter = message_number

            nonce = padding + message_number_in_bytes

            # decrypt message with CCM mode
            cipher = AES.new(self.__session_key, AES.MODE_CCM, nonce)
            decrypted_message_bytes = cipher.decrypt_and_verify(cipher_text, tag)
            decrypted_message = decrypted_message_bytes.decode()

            # print out the result
            print(decrypted_message)

            return decrypted_message
        except ValueError as e:
            print(f"Error:{e}")


# Example
alice = Peer("very secret key!")
bob = Peer("very secret key!")

msg1 = alice.send("msg from alice to bob")
bob.receive(msg1)

msg2 = alice.send("Another msg from alice to bob")
bob.receive(msg2)

msg3 = bob.send("Hello alice")
alice.receive(msg3)


bob.receive(msg1)

msg from alice to bob
Another msg from alice to bob
Hello alice
Error:Message order incorrect
